### Step 1: Setup local configuration

Configure workspace and the resource group

In [ ]:
# If needed
#!az login
!az login --tenant 16b3c013-d300-468d-ac64-7eda0820b6d3

In [ ]:
#!az configure --defaults workspace=<your workspace> group=<your resource group>
!az configure --defaults workspace=gpu-workspace-westus2 group=antonslutsky-rg

### Step 2: Start the training job

In [ ]:
%%writefile anemoi_training_job.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json

code: ./src
command: >-
  echo ${{inputs.data}} ${{outputs.output_dir}}; ./training_driver.sh ${{inputs.data}} ${{outputs.output_dir}}
environment: azureml:anemoihouse:16
resources:
  instance_count: 1
inputs:
  data: 
    type: uri_folder
    mode: ro_mount
    path: azureml:azureml_great_iron_rzt62jgp3b_output_data_data:1
outputs:
  output_dir: 
    type: uri_folder
    mode: upload

distribution:
  type: pytorch
  process_count_per_instance: 2
services:
    my_vs_code:
      type: vs_code
    my_jupyter_lab:
      type: jupyter_lab
compute: azureml:Standard-NC80adis-H100-v5
display_name: anemoihouse
experiment_name: anemoihouse
description: anemoihouse


In [ ]:
!az ml job create -f anemoi_training_job.yaml